# Version 3

## Test Coverage

1. Train에서 공급유형이 장기전세, 공공분양, 공공임대(5년)인 example을 아예 Drop한다.
2. 각 단지코드 내에 임대건물구분 -> '아파트'이면 1, '상가&아파트'이면 0 으로 매핑
3. '자격유형별 평균 임대료' feature 추가

## Import Module

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

## Data Load

In [2]:
DATA_ROOT = ''
DATA_ROOT = os.path.join(DATA_ROOT, "../../../competition_data/parking_data/")

TRAIN_ROOT = os.path.join(DATA_ROOT, "train.csv")
TEST_ROOT = os.path.join(DATA_ROOT, "test.csv")
AGE_GENDER_INFO_ROOT = os.path.join(DATA_ROOT, "age_gender_info.csv")

print(f"DATA_ROOT : {DATA_ROOT}")
print(f"TRAIN_ROOT : {TRAIN_ROOT}")
print(f"TEST_ROOT : {TEST_ROOT}")
print(F"SUBMISSION_ROOT : {AGE_GENDER_INFO_ROOT}")

DATA_ROOT : ../../../competition_data/parking_data/
TRAIN_ROOT : ../../../competition_data/parking_data/train.csv
TEST_ROOT : ../../../competition_data/parking_data/test.csv
SUBMISSION_ROOT : ../../../competition_data/parking_data/age_gender_info.csv


In [4]:
train = pd.read_csv(TRAIN_ROOT)
test = pd.read_csv(TEST_ROOT)
age_gender_info = pd.read_csv(AGE_GENDER_INFO_ROOT)

print("Data Loaded!")

Data Loaded!


## Preprocessing (Version 2)

### `'-'` -> NULL, dtype을 float으로 변경

In [5]:
columns = ['임대보증금', '임대료']

for col in columns:
    train.loc[train[col] == '-', col] = np.nan
    test.loc[test[col] == '-', col] = np.nan

    train[col] = train[col].astype(float)
    test[col] = test[col].astype(float)

### NULL 값 처리

#### 임대보증금, 임대료

In [6]:
train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

#### 지하철, 버스

In [7]:
cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### 자격유형

In [8]:
test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'

#### 중복 example 제거

In [9]:
train = train.drop_duplicates()
test = test.drop_duplicates()

### 단지코드 별 집계

- 단지코드 별로 값이 모두 같은 feature 병합

In [10]:
unique_cols = ['총세대수', '지역', '공가수', \
    '도보 10분거리 내 지하철역 수(환승노선 수 반영)', 
    '도보 10분거리 내 버스정류장 수', 
    '단지내주차면수', '등록차량수']

train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col != '등록차량수']].drop_duplicates()

### 자격유형 병합

- 'J', 'L', 'K', 'N', 'M', 'O' -> '행복주택_공급대상'
    - 공급유형이 행복주택인 경우에서만 나타남
- 'H', 'B', 'E', 'G' -> '국민임대_공급대상'
    - E는 영구임대인 경우도 있긴 하지만 국민임대인 경우가 더 많아서 여기에 포함시킴
- 'C', 'I', 'F', 'G' -> '영구임대_공급대상'

In [11]:
train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

## Preprocessing (Version 3)

### [Test 1] Train에서 공급유형이 장기전세, 공공분양, 공공임대(5년)인 example을 아예 Drop한다.

In [12]:
train_test_1 = pd.DataFrame(train.공급유형)

idx = train_test_1[(train_test_1.공급유형 == '장기전세') | (train_test_1.공급유형 == '공공분양') | (train_test_1.공급유형 == '공공임대(5년)')].index
train_test_1 = train_test_1.drop(idx)

train_test_1.공급유형.unique().tolist()

['국민임대', '공공임대(50년)', '영구임대', '임대상가', '공공임대(10년)', '공공임대(분납)', '행복주택']

### [Test 2] 각 단지코드 내에 임대건물구분 -> '아파트'이면 1, '상가&아파트'이면 0 으로 매핑

In [13]:
train_test_2 = train[['단지코드', '임대건물구분']].copy()

codes = train_test_2.단지코드.unique().tolist()

for code in tqdm(codes):
    values = train_test_2.loc[train_test_2.단지코드 == code, '임대건물구분'].unique().tolist()

    if '상가' in values:
        train_test_2.loc[train_test_2.단지코드 == code, '임대건물구분'] = 0
    else:
        train_test_2.loc[train_test_2.단지코드 == code, '임대건물구분'] = 1

print(f"['임대건물구분' -> 0 (상가 & 아파트)] : {len(train_test_2.loc[train_test_2.임대건물구분 == 0, '단지코드'].unique().tolist())}")
print(f"['임대건물구분' -> 1 (only 아파트)] : {len(train_test_2.loc[train_test_2.임대건물구분 == 1, '단지코드'].unique().tolist())}")

  0%|          | 0/423 [00:00<?, ?it/s]

['임대건물구분' -> 0 (상가 & 아파트)] : 33
['임대건물구분' -> 1 (only 아파트)] : 390


In [14]:
test_test_2 = test[['단지코드', '임대건물구분']].copy()

codes = test_test_2.단지코드.unique().tolist()

for code in tqdm(codes):
    values = test_test_2.loc[test_test_2.단지코드 == code, '임대건물구분'].unique().tolist()

    if '상가' in values:
        test_test_2.loc[test_test_2.단지코드 == code, '임대건물구분'] = 0
    else:
        test_test_2.loc[test_test_2.단지코드 == code, '임대건물구분'] = 1

print(f"['임대건물구분' -> 0 (상가 & 아파트)] : {len(test_test_2.loc[test_test_2.임대건물구분 == 0, '단지코드'].unique().tolist())}")
print(f"['임대건물구분' -> 1 (only 아파트)] : {len(test_test_2.loc[test_test_2.임대건물구분 == 1, '단지코드'].unique().tolist())}")

  0%|          | 0/150 [00:00<?, ?it/s]

['임대건물구분' -> 0 (상가 & 아파트)] : 7
['임대건물구분' -> 1 (only 아파트)] : 143


### [Test 3] '자격유형별 평균 임대료' feature 추가

In [15]:
train_test_3 = train[['자격유형', '임대료']].copy()

qualifies = train_test_3.자격유형.unique().tolist()

for qualify in tqdm(qualifies):
    train_test_3.loc[train_test_3.자격유형 == qualify, '평균임대료(자격유형)'] = train_test_3.loc[train_test_3.자격유형 == qualify, '임대료'].mean()

train_test_3

  0%|          | 0/5 [00:00<?, ?it/s]

,자격유형,임대료,평균임대료(자격유형)
0,A,103680.0,211464.997183
1,A,103680.0,211464.997183
2,A,184330.0,211464.997183
3,A,184330.0,211464.997183
4,A,184330.0,211464.997183
...,...,...,...
2947,A,116090.0,211464.997183
2948,A,142310.0,211464.997183
2949,A,142310.0,211464.997183
2950,A,142310.0,211464.997183


In [16]:
test_test_3 = test[['자격유형', '임대료']].copy()

qualifies = test_test_3.자격유형.unique().tolist()

for qualify in tqdm(qualifies):
    test_test_3.loc[test_test_3.자격유형 == qualify, '평균임대료(자격유형)'] = test_test_3.loc[test_test_3.자격유형 == qualify, '임대료'].mean()

test_test_3

  0%|          | 0/5 [00:00<?, ?it/s]

,자격유형,임대료,평균임대료(자격유형)
0,국민임대_공급대상,189840.0,202304.563107
1,A,249930.0,192464.956063
2,국민임대_공급대상,249930.0,202304.563107
3,국민임대_공급대상,249930.0,202304.563107
4,국민임대_공급대상,296780.0,202304.563107
...,...,...,...
1017,행복주택_공급대상,0.0,109910.330579
1018,국민임대_공급대상,106400.0,202304.563107
1019,국민임대_공급대상,106400.0,202304.563107
1020,국민임대_공급대상,144600.0,202304.563107
